In [5]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly
import plotly.plotly as py
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost:5432/')
plotly.plotly.sign_in('alexderbes', 'jd1okjy9w5')

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()

In [6]:
inverters=pd.read_sql_query(""" select date_trunc('month', "App Received Date"::date) as dte, us_inverter, count(1) from gosolarview where "App Received Date" > '2013-01-01' group by date_trunc('month', "App Received Date"::date), us_inverter  order by dte""", con=engine)
inverters['dte'] = pd.to_datetime(inverters['dte'], utc=True)

In [9]:
# Inverters pct share
totals = inverters.groupby('dte', as_index=False).sum()
frm = pd.merge(inverters, totals, how='left', on='dte')
frm['pct'] = frm['count_x'] / frm['count_y']
pivot = frm.pivot(index='dte', columns='us_inverter', values='pct')
py.iplot([{
    'x': pivot.index,
    'y': pivot[col],
    'name': col
}  for col in pivot.columns], filename='solar/inverters_pct')

In [ ]:
pivot = frm.pivot(index='dte', columns='us_inverter', values='count_x')
py.iplot([{
    'x': pivot.index,
    'y': pivot[col],
    'name': col
}  for col in pivot.columns], filename='solar/inverters_abs')


In [17]:
# Installers 
installers=pd.read_sql_query(""" select date_trunc('month', "App Received Date"::date) as dte, us_installer, count(1) from gosolarview where "App Received Date" > '2013-01-01' and "Customer Sector" = 'Residential' group by date_trunc('month', "App Received Date"::date), us_installer  order by dte""", con=engine)
installers['dte'] = pd.to_datetime(installers['dte'], utc=True)
totals = installers.groupby('dte', as_index=False).sum()
frm = pd.merge(installers, totals, how='left', on='dte')
frm['pct'] = frm['count_x'] / frm['count_y']
pivot = frm.pivot(index='dte', columns='us_installer', values='pct')
py.iplot([{
    'x': pivot.index,
    'y': pivot[col],
    'name': col
}  for col in pivot.columns], filename='solar/inverters')

In [ ]:
installers

In [ ]:
# Installers 
pivot = installers.pivot(index='dte', columns='us_installer', values='count')
py.iplot([{
    'x': pivot.index,
    'y': pivot[col],
    'name': col
}  for col in pivot.columns], filename='solar/inverters')


In [12]:
installer_inverter=pd.read_sql_query("""select date_trunc('month', "App Received Date"::date) as dte, us_installer, us_inverter, count(1) from gosolarview where "App Received Date" > '2013-01-01' group by date_trunc('month', "App Received Date"::date), us_installer, us_inverter  order by dte""", con=engine)
installer_inverter['dte'] = pd.to_datetime(installer_inverter['dte'], utc=True)


In [16]:
#i = 'SCTY'
#frm = installer_inverter[ installer_inverter['us_installer'] == i]
#frm.pivot(index='dte', columns='us_inverter', values='count')
#installer_inverter['us_installer'].unique()

installers = [u'OTHER', u'RUN', u'SCTY', u'Sungevity', u'Sunpower', u'VERENGO',
       u'VSLR']

installers = ['VSLR']

#installer_inverter = installer_inverter.set_index('dte')
for i in installers:
    frm = installer_inverter[ installer_inverter['us_installer'] == i]
    pivot = frm.pivot(index='dte', columns='us_inverter', values='count')
    plotly.offline.iplot([{
    'x': pivot.index,
    'y': pivot[col],
    'name': col
}  for col in pivot.columns])
    

In [ ]:
frm

In [ ]:
installer_inverter.pivot(index='dte', columns='us_installer', values='count')

In [ ]:
installer_inverter.scatter_matrix(filename='test')